# Markov Decision Process

## Bellman equation


<img style="float: right; width: 200px;" src="im/RL_Markov_Bellman.png">

Consider first a model in which there are no actions.
At discrete moments of time, the environment randomly passes from state $s$ to the next state $s'$ with probabilities $P_{ss'}=P(s'|s)$. Upon entering the state with the number $s'$, a fixed reward $r_{s'}$ is charged.

The value $V(s)$ of the state $s$  is the average total discounted reward

$$
R_t = r_t + \gamma\,r_{t+1} + \gamma^2\,r_{t+2}  + ... ~=~ r_t + \gamma\, R_{t+1},
$$


when starting from state $s$ in all of the following states:

$$
V(s) ~=~ \langle R_{t+1}\,|\, s_t = s  \rangle ~=~ \langle r_{t+1}+\gamma\,R_{t+2}\,|\, s_t = s  \rangle.
$$

When moving to the $j$-th state, the agent will receive a reward $r_j$ (for getting into it) plus a discounted future reward equal to $V_j$ for the entire subsequent history (the value of the $j$-th state).

$$
V_i = \sum_j P_{ij}\,\cdot\, \bigr(r_{j} + \gamma\,V_j\bigr)
$$


In [2]:
import numpy as np
from   numpy.linalg import inv     # inverse matrix

## Fake terminal state


<img style="float: right; width: 300px;" src="im/RL_Markov.png">

Let the environment be in five states, two of which (3 and 4) are terminal (after them, the "random walk" ends). It is convenient to introduce one more state (a square in the figure) upon entering which there is no reward and where the environment remains "forever" and the rewards cease to be accrued (end of walk). The transition probabilities are indicated next to the arrows, and the rewards are next to the positions.

In matrix notation, the Bellman equation has a simple solution:

$$
\mathbf{V} = (\mathbf{1} -\gamma\,\mathbf{P})^{-1}\cdot \mathbf{P}\cdot \mathbf{r},
$$

In [16]:
P = np.array([ [0.,   0.6,  0.,   0.4,  0.,   0.,   ],
               [0.,   0.4,  0.5,  0.,   0.1,  0.,   ],
               [0.7,  0.,   0.,   0.2,  0.1,  0.,   ],
               [0.,   0.,   0.,   0.,   0.,   1.0,  ],
               [0.,   0.,   0.,   0.,   0.,   1.0,  ],
               [0.,   0.,   0.,   0.,   0.,   1.0,  ]  ])
r = np.array(  [2.,  -1.,   1.,  -4.,   8.,   0. ] )

In [11]:
gamma = 0.9
np.dot( inv(  np.eye(len(r)) - gamma*P  ), np.dot(P,r) )

array([-1.19488201,  1.86132961,  0.64722433,  0.        ,  0.        ,
        0.        ])

## No fake terminal state


<img src = "im/RL_Markov2.png"  style="float: right; width: 250px; margin-left:10px;">

It is not necessary to introduce a fake terminal state. However, then the rewards will be the matrix $r_{ij}$.
For example, upon transition from a terminal state to it, the reward will be zero $r_{4,4}=0$.
At the same time $r_{1,4}=8$. The Bellman equation in this case takes the following form:

$$
V_i = \sum_j P_{ij}\,\cdot\, \bigr(r_{ij} + \gamma\,V_j\bigr).
$$

More generally, rewards can be probabilistic $P(j,r|i)$ and then:

$$
V_i = \sum_{j,\,r} P_{ijr}\,\cdot\, \bigr(r + \gamma\,V_j\bigr).
$$



In [7]:
P = np.array([ [0.,   0.6,  0.,   0.4,  0.  ],
               [0.,   0.4,  0.5,  0.,   0.1 ],
               [0.7,  0.,   0.,   0.2,  0.1 ],
               [0.,   0.,   0.,   1.0,  0.  ],
               [0.,   0.,   0.,   0.,   1.0 ] ])

r = np.array([ [0.,  -1.,   0.,  -4.,   0.  ],
               [0.,  -1.,   1.,   0.,   8.  ],
               [2.,   0.,   0.,  -4.,   8.  ],
               [0.,   0.,   0.,   0.,   0.  ],
               [0.,   0.,   0.,   0.,   0.  ] ])

In [5]:
gamma = 0.9
np.dot( inv(  np.eye(len(r)) - gamma*P  ), np.sum(P*r, axis=1) )

array([-1.19488201,  1.86132961,  0.64722433,  0.        ,  0.        ])

## Monte Carlo simulation

Let's reproduce the same result using Monte Carlo simulation. To do this, we will start walking the environment many times starting from the same state <b class="norm">s0</b>, calculating the reward received for each episode:

In [22]:
states      = np.arange(len(r))             # [0,1,2,3,4,5]
terminals   = [3,4]                         # terminal states
gamma       = 0.9                           # discount factor
rews, steps = [], []                        # rewards and steps
s0          = 0                             # start state

for _ in range(10000):                      # run 10,000 experiments
    rew, s, discont = 0, s0, 1
    for i in range(100):                    # just in case, limit
        s = np.random.choice(states,p=P[s]) # state by probability distribution P[s]

        rew += r[s] * discont               # accumulating reward
        discont *= gamma                    # we discount more

        if s in terminals:                  # entered the terminal state
            break                           # further reward does not change

    rews.append(rew)                        # keep the reward
    steps.append(i+1)                       # save the number of steps taken

print("V(%d) =  %7.2f ± %5.4f  T(%d): %7.2f ± %5.4f"
      % (s0, np.mean(rews),  np.std(rews) /len(rews)**0.5,
         s0, np.mean(steps), np.std(steps)/len(steps)**0.5))

V(0) =    -1.19 ± 0.0365  T(0):    3.85 ± 0.0366


## Policy function

Let an agent with probability $\pi_{i\alpha}=\pi(\alpha\,|\,i)$ in a state with number $i$ perform an action with number $\alpha$ (below, Latin indices are used for state numbers, and Greek for action).
As a result of the agent's action, the environment passes into the state numbered $j$ with the probability $P_{i\alpha j}=P(j\,|\,i,\alpha)$.

The state values still satisfy the Bellman equation:

$$
V_i = \sum_j \tilde{P}_{ij}\, \bigr(r_j + \gamma\,V_j\bigr),~~~~~~~~~~~~\tilde{P}_{ij} = \sum_\alpha \pi_{i\alpha}\,P_{i\alpha j},
$$

where $\tilde{P}_{ij}$ the probabilities of going from $i$ to $j$ are equal to the product of the probability of taking the action $\alpha$ in state $i$ by the probability of going after that to state $j$ with the sum over all possible actions.

Let us write down the equations for the Q-utility function of the action $a$ performed in the state $s$:

$$
Q(s,a) ~=~ \langle R_{t+1}\,|\,s_t=a,\,a_t=a\rangle.
$$

For the discrete case, this will be the $Q_{i\alpha}$ matrix.
Knowing it, we can calculate the utility of the state (when performing any action with probabilities $\pi_{i\alpha}$):

$$
V_i = \sum_\alpha \pi_{i\alpha} \, Q_{i\alpha}.
$$


You can also record feedback. The value of actions $Q_{i\alpha}$ in state $i$ is equal to the average reward for the immediate transition, plus the discounted future rewards of the states that the environment enters during such a transition:

$$
Q_{i\alpha} = \sum_{j}P_{i\alpha j}\,(r_j + \gamma\,V_j\bigr).
$$

In [27]:
num_states, num_actions = 10, 3                            # number of states and actions

pi = np.random.random((num_states, num_actions))           # probabilities of action
pi = pi/np.sum(pi, axis=1).reshape(-1,1)                   # normalize to unit

P = np.random.random((num_states,num_actions,num_states))  # environment model
P = P/np.sum(P, axis=2).reshape(num_states,num_actions,1)  # normalize

r  = np.linspace(-10, 10, num_states)                      # rewards [-10...10]

piP = np.sum( np.expand_dims(pi, axis=2) * P, axis=1)

V = np.dot( inv(  np.eye(len(r)) - gamma*piP  ), np.dot( piP, r) )
Q = np.dot(P, r + gamma*V)
